In [1]:
import pandas as pd
import numpy as np
import subprocess
import random
import pipeline_script
import seaborn as sns
import math
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
from numpy.random import RandomState
from sklearn.preprocessing import MinMaxScaler
import os

Import Processed Participant DF

In [2]:
participant_df = pd.read_csv("participant_info_processed.csv",index_col="SID")
participant_df["GENDER"] = participant_df["GENDER"].apply(lambda x: 1 if x == "M" else 0)
participant_df.head()

,AGE,GENDER,BMI,OAHI,AHI,Mean_SaO2,Arousal Index,MEDICAL_HISTORY,Sleep_Disorders,asthma,...,eds,rbd,hypersomnia,grind teeth,insomnia,fatigue,morning headaches,bruxism,snore,headache
SID,,,,,,,,,,,,,,,,,,,,,
S002,0.679909,1,0.166667,19,19,91%,98,"Asthma, Body Pain, GERD, Hypertension, Sleep A...",OSA,0,...,0,0,0,0,0,0,0,0,0,0
S003,0.124049,0,0.611111,34,37,95%,28,NaN,"snoring, sleep apnea, difficulty breathing, sn...",0,...,0,0,0,0,0,0,0,0,0,0
S004,0.524049,0,0.425926,63,99,89%,109,"Arrhythmia, Body Pain, Depression, Dyspnea, GERD",difficulty breathing,0,...,0,0,0,0,0,0,0,0,0,0
S005,0.424505,0,0.462963,19,20,95%,28,"Asthma, Body Pain, Depression, Diabetes, Dyspn...",OSA,0,...,0,0,0,0,0,0,0,0,0,0
S006,0.238661,0,0.074074,4,5,97%,34,"Depression, Sleep Apnea",OSA,0,...,0,0,0,0,0,0,0,0,0,0


Import Processed Patient Records

In [3]:

def extract_sxxx(filename):
    # Define the pattern to match 'Sxxx' where x is a digit (1 to 3 digits)
    pattern = r"(S\d{1,3})"
    match = re.search(pattern, filename)
    if match:
        return match.group(1)  # Return the matched string
    return None  #

def gather_dict(parent_folder,process = True):
    csv_files = [file for file in os.listdir(parent_folder) if file.endswith(".parquet")]
    patients_records={}
    for patient in tqdm(csv_files):
        patient_id = extract_sxxx(patient)
        if not patient_id:
            return
        patients_records[patient_id] = pd.read_parquet(os.path.join(parent_folder, patient))

    return patients_records

In [4]:
patient_records = gather_dict("normalized_data")

 58%|█████▊    | 57/98 [00:35<00:25,  1.60it/s]


MemoryError: 

In [3]:
patient_records = pipeline_script.gather_dict("normalized_data_20")

100%|██████████| 98/98 [00:06<00:00, 15.57it/s]


Specify The Columns To Use In Training And Prediction

In [4]:
patient_records["S002"].head(1).columns

Index(['', 'TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR',
       'IBI', 'IBI_Moving_Average_640', 'IBI_Moving_STD_640', 'ACC',
       'Abnormal_IBI_Moving_4_Mean_Scaled', 'HR_Divided_Mean',
       'HR_Divided_Median', 'Sleep_Stage', 'Abnormal_IBI_Moving_3',
       'Abnormal_IBI_Moving_4'],
      dtype='object')

In [23]:
columns = [ 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR',
       'IBI',
       'Multiple_Events', 'IBI_Moving_Average_640', 'IBI_Moving_STD_640',
       'Abnormal_IBI_Moving_3', 'Abnormal_IBI_Moving_4', 'ACC',
       'HR_Divided_Mean', 'HR_Divided_Median',
       'Abnormal_IBI_Moving_4_Mean_Scaled']

In [6]:
participant_df.columns

Index(['AGE', 'GENDER', 'BMI', 'OAHI', 'AHI', 'Mean_SaO2', 'Arousal Index',
       'MEDICAL_HISTORY', 'Sleep_Disorders', 'asthma', 'body pain', 'gerd',
       'hypertension', 'sleep apnea', 'arrhythmia', 'depression', 'dyspnea',
       'diabetes', 'cad', 'anxiety', 'migraine', 'osa', 'snoring',
       'difficulty breathing', 'difficulty staying asleep', 'snort', 'rls',
       'eds', 'rbd', 'hypersomnia', 'grind teeth', 'insomnia', 'fatigue',
       'morning headaches', 'bruxism', 'snore', 'headache'],
      dtype='object')

In [7]:
columns_participant=['AGE', 'GENDER', 'BMI','asthma',
       'body pain', 'gerd', 'hypertension', 'sleep apnea', 'arrhythmia',
       'depression', 'dyspnea', 'diabetes', 'cad', 'anxiety', 'migraine',
       'osa', 'snoring', 'difficulty breathing', 'difficulty staying asleep',
       'snort', 'rls', 'eds', 'rbd', 'hypersomnia', 'grind teeth', 'insomnia',
       'fatigue', 'morning headaches', 'bruxism', 'snore', 'headache']

Dimensionality =

In [8]:
print(len(columns)+len(columns_participant))

48


Creating Model

1. Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

In [10]:
rng = np.random.default_rng(seed = 30)
test_validation_indexes = rng.random(30)
a, b = 0, len(patient_records.keys())  # Example range
test_validation_indexes = np.floor(a + (b - a) * test_validation_indexes)
test_validation_indexes = [int(i) for i in test_validation_indexes]

validation_indexes = test_validation_indexes[:15]
test_indexes = test_validation_indexes[15:]
print(validation_indexes,test_indexes)

[23, 42, 9, 58, 76, 85, 31, 10, 39, 57, 24, 63, 88, 38, 46] [75, 59, 72, 49, 86, 15, 58, 9, 94, 42, 11, 88, 91, 23, 31]


In [46]:
for index in test_indexes:  # test_indexes içindeki indeksler
    key = f"S{index:03}"  # Sayıları 'SXXX' formatına dönüştür (örneğin 76 -> 'S076')
    
    if key in patient_records:  # Anahtar sözlükte var mı kontrol et
        print(f"Test - Index {key}:")
        df = patient_records[key]  # Sözlükten doğru anahtara göre veri al
        print(df["Sleep_Stage"].value_counts())  # Sleep_Stage sınıf dağılımını yazdır
    else:
        print(f"Key '{key}' not found in patient_records!")

Test - Index S075:
Sleep_Stage
N2    39841
P     29184
W     19392
R      6336
N1     4128
N3      192
Name: count, dtype: int64
Test - Index S059:
Sleep_Stage
N1    29088
W     23520
N2    23136
P     20964
R      2784
N3      288
Name: count, dtype: int64
Test - Index S072:
Sleep_Stage
W     42048
P     29847
N2    15456
N1    12864
Name: count, dtype: int64
Test - Index S049:
Sleep_Stage
N2    52416
P     30957
W     13536
R      8160
N1     2112
Name: count, dtype: int64
Test - Index S086:
Sleep_Stage
N2    37249
P     22608
W     22368
N1    12576
R      7680
Name: count, dtype: int64
Test - Index S015:
Sleep_Stage
N2    52512
P     27002
R     18240
N3     7200
N1     6912
W      6816
Name: count, dtype: int64
Test - Index S058:
Sleep_Stage
W     26496
N2    26112
P     24580
N1    23616
Name: count, dtype: int64
Test - Index S009:
Sleep_Stage
N2    39264
W     24192
P     20941
R      7296
N1     6432
N3      288
Name: count, dtype: int64
Test - Index S094:
Sleep_Stage
N2    512

In [10]:
train_set = {}

In [11]:
for i,v in tqdm(patient_records.items()):
    train_set[i] = v.loc[v["Sleep_Stage"]!="P"].copy().drop("",axis = 1)
    for column in columns_participant:
        train_set[i].loc[:,column] = participant_df.loc[i,column]

100%|██████████| 98/98 [00:03<00:00, 30.63it/s]


In [12]:
train_set["S002"]["GENDER"]

29357     1
29358     1
29359     1
29360     1
29361     1
         ..
100680    1
100681    1
100682    1
100683    1
100684    1
Name: GENDER, Length: 71328, dtype: int64

In [18]:
list(train_set.keys())

['S002',
 'S003',
 'S004',
 'S005',
 'S006',
 'S007',
 'S008',
 'S009',
 'S010',
 'S011',
 'S012',
 'S013',
 'S014',
 'S015',
 'S016',
 'S017',
 'S018',
 'S019',
 'S020',
 'S021',
 'S022',
 'S023',
 'S024',
 'S025',
 'S026',
 'S028',
 'S029',
 'S030',
 'S031',
 'S032',
 'S033',
 'S034',
 'S035',
 'S036',
 'S037',
 'S038',
 'S039',
 'S040',
 'S042',
 'S043',
 'S044',
 'S045',
 'S046',
 'S047',
 'S048',
 'S049',
 'S050',
 'S051',
 'S052',
 'S053',
 'S054',
 'S055',
 'S056',
 'S057',
 'S058',
 'S059',
 'S061',
 'S062',
 'S063',
 'S064',
 'S065',
 'S066',
 'S067',
 'S068',
 'S069',
 'S070',
 'S071',
 'S072',
 'S073',
 'S074',
 'S075',
 'S076',
 'S078',
 'S079',
 'S080',
 'S081',
 'S082',
 'S083',
 'S084',
 'S085',
 'S086',
 'S087',
 'S088',
 'S089',
 'S090',
 'S091',
 'S092',
 'S093',
 'S094',
 'S095',
 'S096',
 'S097',
 'S098',
 'S099',
 'S100',
 'S101',
 'S102',
 'S103']

In [13]:
filtered_dict = {}
for i,key in tqdm(enumerate(list(train_set.keys()))):
    if i in test_indexes or i in validation_indexes:
        continue
    else:
        filtered_dict[key] = train_set[key]
len(filtered_dict.keys())

0it [00:00, ?it/s]


NameError: name 'test_indexes' is not defined

In [14]:
combined_df = pd.concat(train_set.values(),ignore_index=True)
combined_df.head()

,TIMESTAMP,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,IBI_Moving_Average_640,...,eds,rbd,hypersomnia,grind teeth,insomnia,fatigue,morning headaches,bruxism,snore,headache
0,9174.210938,7.1840,-21.3,-61.5,6.3,32.09,0.065832,72.85,1.0625,0.912887,...,0,0,0,0,0,0,0,0,0,0
1,9174.523438,-3.3475,-21.3,-61.4,6.0,32.09,0.065064,72.85,1.0625,0.912985,...,0,0,0,0,0,0,0,0,0,0
2,9174.835938,-12.6025,-21.4,-61.5,5.3,32.09,0.065064,72.85,1.0625,0.913082,...,0,0,0,0,0,0,0,0,0,0
3,9175.148438,-8.7865,-20.9,-61.7,5.5,32.11,0.065320,73.33,1.0625,0.913180,...,0,0,0,0,0,0,0,0,0,0
4,9175.460938,-1.1710,-21.6,-61.3,5.3,32.11,0.065320,73.33,1.0625,0.913278,...,0,0,0,0,0,0,0,0,0,0


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [17]:

# Adım 3: Yaygın hastaları stratify ile böl
train_common, test_val_common = train_test_split(
    combined_df, test_size=0.3, random_state=42, stratify=combined_df["Sleep_Stage"]
)


In [18]:
test_val_common.columns

Index(['TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR',
       'IBI', 'IBI_Moving_Average_640', 'IBI_Moving_STD_640', 'ACC',
       'Abnormal_IBI_Moving_4_Mean_Scaled', 'HR_Divided_Mean',
       'HR_Divided_Median', 'Sleep_Stage', 'Abnormal_IBI_Moving_3',
       'Abnormal_IBI_Moving_4', 'AGE', 'GENDER', 'BMI', 'asthma', 'body pain',
       'gerd', 'hypertension', 'sleep apnea', 'arrhythmia', 'depression',
       'dyspnea', 'diabetes', 'cad', 'anxiety', 'migraine', 'osa', 'snoring',
       'difficulty breathing', 'difficulty staying asleep', 'snort', 'rls',
       'eds', 'rbd', 'hypersomnia', 'grind teeth', 'insomnia', 'fatigue',
       'morning headaches', 'bruxism', 'snore', 'headache'],
      dtype='object')

In [19]:

# Test ve validation setini oluştur
validation_common, test_common = train_test_split(
    test_val_common, test_size=0.5, random_state=42, stratify=test_val_common["Sleep_Stage"]
)


In [ ]:
def eval_model(model,validation_set):
    y_pred = []
    y_test = []
    for i in validation_indexes:
        index = list(train_set.keys())[int(i)]
        df_pred = train_set[index]
        y_pred.extend(list(model.predict(df_pred[cols])))
        y_test.extend(list(df_pred["Sleep_Stage"]))

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    return accuracy,cm

In [20]:
cols = combined_df.columns.to_list()
cols.remove("Sleep_Stage")
cols

['TIMESTAMP',
 'BVP',
 'ACC_X',
 'ACC_Y',
 'ACC_Z',
 'TEMP',
 'EDA',
 'HR',
 'IBI',
 'IBI_Moving_Average_640',
 'IBI_Moving_STD_640',
 'ACC',
 'Abnormal_IBI_Moving_4_Mean_Scaled',
 'HR_Divided_Mean',
 'HR_Divided_Median',
 'Abnormal_IBI_Moving_3',
 'Abnormal_IBI_Moving_4',
 'AGE',
 'GENDER',
 'BMI',
 'asthma',
 'body pain',
 'gerd',
 'hypertension',
 'sleep apnea',
 'arrhythmia',
 'depression',
 'dyspnea',
 'diabetes',
 'cad',
 'anxiety',
 'migraine',
 'osa',
 'snoring',
 'difficulty breathing',
 'difficulty staying asleep',
 'snort',
 'rls',
 'eds',
 'rbd',
 'hypersomnia',
 'grind teeth',
 'insomnia',
 'fatigue',
 'morning headaches',
 'bruxism',
 'snore',
 'headache']

In [32]:
train_common[cols]

,TIMESTAMP,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,IBI_Moving_Average_640,...,eds,rbd,hypersomnia,grind teeth,insomnia,fatigue,morning headaches,bruxism,snore,headache
2763650,19130.148438,-12.4690,29.0,13.0,-54.0,29.530,0.130403,48.380,1.250000,1.265963,...,0,0,0,0,0,0,0,0,0,0
152231,6303.585938,-25.4255,-39.2,47.1,19.8,32.890,0.236245,93.380,0.637500,0.639924,...,0,0,0,0,0,0,0,0,0,0
7055442,23781.398438,72.4150,12.5,-52.9,31.3,32.000,0.348585,65.530,1.031250,1.031250,...,0,0,0,0,0,0,0,0,0,0
6116065,18400.148438,-43.8910,36.9,-43.8,26.2,31.690,0.274951,66.480,0.890625,0.898917,...,1,0,0,0,0,0,0,0,0,0
2440736,10214.523438,-78.9050,-64.0,11.0,-2.0,33.680,0.522924,47.180,1.275000,1.260592,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279042,24093.898438,12.3595,-60.9,-0.2,-23.0,35.474,0.592416,70.916,0.875000,0.853413,...,0,0,0,0,0,0,0,0,0,0
7271574,19725.148438,6.2045,49.0,-16.9,35.1,35.130,0.146010,66.730,0.843750,0.900206,...,1,0,0,0,0,0,0,0,0,0
4193034,10002.023438,5.7555,31.0,12.0,-53.6,33.642,0.037916,67.764,1.265625,1.265625,...,0,0,0,0,0,0,0,0,0,0
4136427,17110.148438,4.8600,-16.0,56.0,28.0,35.790,0.166297,69.250,0.796875,0.843813,...,1,0,0,0,0,0,0,0,0,0


In [39]:
5279781//32

164993

In [52]:
train_common[cols][:1 ]

,TIMESTAMP,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,IBI_Moving_Average_640,...,eds,rbd,hypersomnia,grind teeth,insomnia,fatigue,morning headaches,bruxism,snore,headache
2763650,19130.148438,-12.469,29.0,13.0,-54.0,29.53,0.130403,48.38,1.25,1.265963,...,0,0,0,0,0,0,0,0,0,0


In [60]:
total_estimators = 2
max_depth=30
rf_classifier = RandomForestClassifier(n_estimators=total_estimators,criterion='gini',max_features='sqrt', warm_start=True, random_state=30, max_depth = max_depth,bootstrap=True,class_weight='balanced',n_jobs=-1,verbose=1)
cols = combined_df.columns.to_list()
cols.remove("Sleep_Stage")
cms = []
# Track progress
with tqdm(total=total_estimators, desc="Training Progress") as pbar:
    for i in range(1, total_estimators + 1):
        rf_classifier.set_params(n_estimators=i)  # Incrementally add trees
        rf_classifier.fit(train_common[cols], train_common["Sleep_Stage"])      # Train the model
        pbar.update(1)  
        y_pred = (list(rf_classifier.predict(validation_common[cols])))
        accuracy = accuracy_score(validation_common["Sleep_Stage"], y_pred)
        cm = confusion_matrix(validation_common["Sleep_Stage"], y_pred)
        print(accuracy,cm)
        print(precision_score(validation_common["Sleep_Stage"], y_pred,average="macro"))
        print(recall_score(validation_common["Sleep_Stage"], y_pred,average="macro"))

        print(f1_score(validation_common["Sleep_Stage"], y_pred,average="macro"))

        cms.append(cm)

Training Progress:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\alkan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
Training Progress:  50%|█████     | 1/2 [00:47<00:47, 47.45s/it]

0.9607639152823715 [[115964   5690     19    629   3597]
 [ 11798 543302   3722   1209   4248]
 [    29   1258  37531     14     48]
 [   481    918     11 117010    582]
 [  4907   4242     88    901 273184]]
0.9406084967727587
0.9593388682820972
0.9496359446709828


Training Progress:  50%|█████     | 1/2 [01:20<01:20, 80.65s/it]


KeyboardInterrupt: 

In [68]:
print(type(validation_common[cols]))

<class 'pandas.core.frame.DataFrame'>


In [92]:
columns

['BVP',
 'ACC_X',
 'ACC_Y',
 'ACC_Z',
 'TEMP',
 'EDA',
 'HR',
 'IBI',
 'IBI_Moving_Average_640',
 'IBI_Moving_STD_640',
 'Abnormal_IBI_Moving_3',
 'Abnormal_IBI_Moving_4',
 'ACC',
 'HR_Divided_Mean',
 'HR_Divided_Median',
 'Abnormal_IBI_Moving_4_Mean_Scaled']

In [95]:
a = 0
for i in test_common.index:
    if i in train_common.index:
        a +=1 
a    

0

In [91]:
y_pred = (list(rf_classifier.predict(test_common[columns])))
accuracy = accuracy_score(test_common["Sleep_Stage"], y_pred)
cm = confusion_matrix(test_common["Sleep_Stage"], y_pred)
print(accuracy,cm)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished


0.9662419645288214 [[ 98709   2644      1    193   1270]
 [  6620 408706    818    222    918]
 [    20   1666  23160      2     21]
 [   657   1153      6  84351    204]
 [  5334   6007     69   1063 211923]]


In [79]:
y_pred = (list(rf_classifier.predict(validation_common[cols])))
accuracy = accuracy_score(validation_common["Sleep_Stage"], y_pred)
cm = confusion_matrix(validation_common["Sleep_Stage"], y_pred)
print(accuracy,cm)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished


0.9720825114287585 [[ 99716   2108      4    123    865]
 [  7193 408535    684    116    755]
 [    20   1572  23255      1     21]
 [   468    785      1  85010    108]
 [  4296   4037     42    691 215330]]


In [ ]:
accs = []
cms = []
for i,key in tqdm(enumerate(list(train_set.keys())[40:])):
    if i in test_indexes or i in validation_indexes:
        continue
    df = train_set[key]
    cols = df.columns.to_list()
    cols.remove("Sleep_Stage")
    rf_classifier.fit(df.loc[:,cols], df["Sleep_Stage"]) #Xtrain and Ytrain

    y_pred = []
    y_test = []
    for i in validation_indexes:
        index = list(train_set.keys())[int(i)]
        df_pred = train_set[index]
        y_pred.extend(list(rf_classifier.predict(df_pred[cols])))
        y_test.extend(list(df_pred["Sleep_Stage"]))

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(accuracy)
    print(cm.ravel())
    accs.append(accuracy)
    cms.append(cm)


In [77]:
combined_df.shape

(5704907, 53)

In [78]:
total_estimators = 100
max_depth=48
rf_regressor = RandomForestClassifier(n_estimators=total_estimators, warm_start=True, random_state=30, max_depth = max_depth)
cols = combined_df.columns.to_list()
cols.remove("Sleep_Stage")
# Track progress
with tqdm(total=total_estimators, desc="Training Progress") as pbar:
    for i in range(1, total_estimators + 1):
        rf_regressor.set_params(n_estimators=i)  # Incrementally add trees
        rf_regressor.fit(combined_df[cols], combined_df["Sleep_Stage"])      # Train the model
        pbar.update(1)
        print(accuracy)
        print(cm)

Training Progress: 100%|██████████| 100/100 [32:35<00:00, 19.55s/it]


In [81]:
y_pred = []
y_test = []
for i in validation_indexes:
    index = list(train_set.keys())[int(i)]
    df_pred = train_set[index]
    y_pred.extend(list(rf_regressor.predict(df_pred[cols])))
    y_test.extend(list(df_pred["Sleep_Stage"]))

accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(accuracy)
print(cm)

0.22711721534690646
[[   300   7504      0      0  75716]
 [  5613  43948      0      0 563016]
 [  2231   2208      0      0  52105]
 [   659  13881      0      0 126773]
 [  2715  22286      0      0 212791]]


In [82]:
np.unique(np.array(y_pred),return_counts=True)

(array(['N1', 'N2', 'W'], dtype='<U2'), array([  11518,   89827, 1030401]))

In [ ]:
accs = []
cms = []
for i,key in tqdm(enumerate(list(train_set.keys()))):
    if i in test_indexes or i in validation_indexes:
        continue
    df = train_set[key]
    pd.concat
    cols = df.columns.to_list()
    cols.remove("Sleep_Stage")
    rf_classifier.fit(df.loc[:,columns], df["Sleep_Stage"]) #Xtrain and Ytrain

    y_pred = []
    y_test = []
    for i in validation_indexes:
        index = list(train_set.keys())[int(i)]
        df_pred = train_set[index]
        y_pred.extend(list(rf_classifier.predict(df_pred[columns])))
        y_test.extend(list(df_pred["Sleep_Stage"]))

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(accuracy)
    print(cm.ravel())
    accs.append(accuracy)
    cms.append(cm)


In [ ]:
np.unique(np.array(y_pred),return_counts=True)

(array(['N1', 'N2', 'R', 'W'], dtype='<U2'),
 array([ 56874, 172526,    186, 902160]))

In [68]:
np.unique(np.array(y_test),return_counts=True)

(array(['N1', 'N2', 'N3', 'R', 'W'], dtype='<U2'),
 array([ 83520, 612577,  56544, 141313, 237792]))

In [67]:
accuracy_score(y_test, y_pred)

0.22762527987728695

In [ ]:
accs = []
cms = []
for i,key in tqdm(enumerate(list(train_set.keys()))):
    if i in test_indexes or i in validation_indexes:
        continue
    df = train_set[key]
    cols = df.columns.to_list()
    cols.remove("Sleep_Stage")
    rf_classifier.fit(df.loc[:,columns], df["Sleep_Stage"]) #Xtrain and Ytrain

    y_pred = []
    y_test = []
    for i in validation_indexes:
        index = list(train_set.keys())[int(i)]
        df_pred = train_set[index]
        y_pred.extend(list(rf_classifier.predict(df_pred[columns])))
        y_test.extend(list(df_pred["Sleep_Stage"]))

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(accuracy)
    print(cm.ravel())
    accs.append(accuracy)
    cms.append(cm)
